## Prediction Model based on Genres


#### Model 1: generate list of movie recommendation based on "genres" + "rating".
- Input from user: genres
- scores are calculated and ranked to produce the top 5 movies
- In this notebook, I will offer 2 simple way:
    - based on rating score alone
    - based on rating score and genre picked by the user
- Features include: 
    - genres
    - movie rating score (based on IMDB formula)

In [ ]:
import pandas as pd

pd.set_option("display.max_columns", 85)
pd.set_option("display.max_rows", 85)
df = pd.read_csv('../../edit_data/Lee/cleaned_data/movies_main.csv')  
df.head()

,Id,Original_Title,Original_Language,Spoken_Languages,Budget,Revenue,Runtime,Release_Date,Production_Companies,Production_Countries,imdb_id,Popularity,Vote_Average,Vote_Count,Belongs_To_Collection,Tagline,Overview,Genres,Genres_Parse,Belongs_To_Collection_Parse,Spoken_Languages_Parse,Production_Companies_Parse,Production_Countries_Parse,Production_Countries_Code_Parse,Keywords,Keywords_parse,Cast_parse,Director_parse
0,862,Toy Story,en,"[{'iso_639_1': 'en', 'name': 'English'}]",30000000.0,373554033.0,81.0,1995-10-30,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0114709,21.946943,7.7,5415.0,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","['Animation', 'Comedy', 'Family']",['Toy Story Collection'],['English'],['Pixar Animation Studios'],['United States of America'],['US'],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","['jealousy', 'toy', 'boy', 'friendship', 'frie...","['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...",['John Lasseter']
1,8844,Jumanji,en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",65000000.0,262797249.0,104.0,1995-12-15,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0113497,17.015539,6.9,2413.0,NaN,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","['Adventure', 'Fantasy', 'Family']",NaN,"['English', 'Français']","['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],['US'],"[{'id': 10090, 'name': 'board game'}, {'id': 1...","['board game', 'disappearance', ""based on chil...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",['Joe Johnston']
2,15602,Grumpier Old Men,en,"[{'iso_639_1': 'en', 'name': 'English'}]",0.0,0.0,101.0,1995-12-22,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0113228,11.712900,6.5,92.0,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","['Romance', 'Comedy']",['Grumpy Old Men Collection'],['English'],"['Warner Bros.', 'Lancaster Gate']",['United States of America'],['US'],"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","['fishing', 'best friend', 'duringcreditssting...","['Walter Matthau', 'Jack Lemmon', 'Ann-Margret...",['Howard Deutch']
3,31357,Waiting to Exhale,en,"[{'iso_639_1': 'en', 'name': 'English'}]",16000000.0,81452156.0,127.0,1995-12-22,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",tt0114885,3.859495,6.1,34.0,NaN,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","['Comedy', 'Drama', 'Romance']",NaN,['English'],['Twentieth Century Fox Film Corporation'],['United States of America'],['US'],"[{'id': 818, 'name': 'based on novel'}, {'id':...","['based on novel', 'interracial relationship',...","['Whitney Houston', 'Angela Bassett', 'Loretta...",['Forest Whitaker']
4,11862,Father of the Bride Part II,en,"[{'iso_639_1': 'en', 'name': 'English'}]",0.0,76578911.0,106.0,1995-02-10,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0113041,8.387519,5.7,173.0,"{'id': 96871, 'name': 'Father of the Bride Col...",Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",['Comedy'],['Father of the Bride Collection'],['English'],"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],['US'],"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","['baby', 'midlife crisis', 'confidence', 'agin...","['Steve Martin', 'Dian

: 

Method 1: step 1 - calculate the C and m values.

In [ ]:
## To calculate the mean value of Vote Average
C= df['Vote_Average'].mean()
C

5.6259312895463855

In [ ]:
## set vote count threshold to 90 %tile.
m= df['Vote_Count'].quantile(0.9)
m

163.0

Method 1: Step 2 - Set qualified Movies (The Filter)
- Qualified movies are set based on Number of Vote Count received by the movies. 
- To qualified, the movies must has be at 90 %tile in total vote counts (i.e. 163 votes or more)
- This is where we would set the ['Genres']' as a filter. 

Filter 1: m-value only

In [ ]:
## only filter based on m value
# qualified_movies = df.copy().loc[df['Vote_Count'] >= m]
# qualified_movies.shape


(4492, 28)

Filter 2: m-value + genres (entered by user)

Re-Parse Genres again to a new column (the parsed column is not working)

In [59]:
import ast
import numpy as np

# Function to parse the string and extract 'name'
def extract_names(data_string):
    if pd.isna(data_string):
        return np.nan
    try:
        # Safely evaluate the string to a Python object
        data_object = ast.literal_eval(data_string)
        if isinstance(data_object, list):
            # Extract 'name' from each dictionary in the list
            names = [item['name'] for item in data_object]
            return names
        elif isinstance(data_object, dict):
            # Extract 'name' from the dictionary
            return [data_object.get('name', np.nan)]
    except (ValueError, SyntaxError):
        return np.nan
    
    
df['Genres_Parse_new'] = df['Genres'].apply(extract_names)

In [63]:
### TEST : to generate a unique list of genres

# Flatten the list of genres, ignoring NaN values, and extract unique values
unique_genres = set(
    genre
    for sublist in df['Genres_Parse_new'].dropna()  # Drop NaN values
    for genre in sublist
)

# Convert the set to a sorted list (optional)
unique_genres_list = sorted(unique_genres)

# Display the unique genres
print(unique_genres_list)
print(len(unique_genres_list))

['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']
20


Request user to enter the Genre
- codes can be enhanced to avoid typo, etc. 

In [93]:

# Genres List in total (20)
genres_list= ['Action', 'Adventure', 'Animation', 
              'Comedy', 'Crime', 'Documentary', 
              'Drama', 'Family', 'Fantasy', 
              'Foreign', 'History', 'Horror', 
              'Music', 'Mystery', 'Romance', 
              'Science Fiction', 'TV Movie', 
              'Thriller', 'War', 'Western']

print('total genres:', len(genres_list))

genre_input = input("Enter the genre: ")

if x in genres_list:
    filt = (df['Vote_Count'] >= m) & (df['Genres_Parse_new'].apply(lambda x: genre_input in x if isinstance(x, list) else False) )
    qualified_movies = df.copy().loc[filt]
    print("You have selected the genre: ", genre_input)
    print("qualified_movies: ", qualified_movies.shape)
else:
    print("wrong genre entered.")
    Game_is_on = False



total genres: 20
You have selected the genre:  Family
qualified_movies:  (538, 30)


Method 1: step 3 - set up the rating score calculation

In [94]:
def weighted_rating(x, m=m, C=C):
    v = x['Vote_Count']
    R = x['Vote_Average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [95]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
qualified_movies['Score'] = qualified_movies.apply(weighted_rating, axis=1)

qualified_movies.head()

,Id,Original_Title,Original_Language,Spoken_Languages,Budget,Revenue,Runtime,Release_Date,Production_Companies,Production_Countries,imdb_id,Popularity,Vote_Average,Vote_Count,Belongs_To_Collection,Tagline,Overview,Genres,Genres_Parse,Belongs_To_Collection_Parse,Spoken_Languages_Parse,Production_Companies_Parse,Production_Countries_Parse,Production_Countries_Code_Parse,Keywords,Keywords_parse,Cast_parse,Director_parse,Genres_Parse_new,Has_Animation,Score
0,862,Toy Story,en,"[{'iso_639_1': 'en', 'name': 'English'}]",30000000.0,373554033.0,81.0,1995-10-30,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0114709,21.946943,7.7,5415.0,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","['Animation', 'Comedy', 'Family']",['Toy Story Collection'],['English'],['Pixar Animation Studios'],['United States of America'],['US'],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","['jealousy', 'toy', 'boy', 'friendship', 'frie...","['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...",['John Lasseter'],"[Animation, Comedy, Family]",True,7.639392
1,8844,Jumanji,en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",65000000.0,262797249.0,104.0,1995-12-15,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0113497,17.015539,6.9,2413.0,NaN,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","['Adventure', 'Fantasy', 'Family']",NaN,"['English', 'Français']","['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],['US'],"[{'id': 10090, 'name': 'board game'}, {'id': 1...","['board game', 'disappearance', ""based on chil...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",['Joe Johnston'],"[Adventure, Fantasy, Family]",True,6.819382
12,21032,Balto,en,"[{'iso_639_1': 'en', 'name': 'English'}]",0.0,11348324.0,78.0,1995-12-22,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0112453,12.140733,7.1,423.0,"{'id': 117693, 'name': 'Balto Collection', 'po...",Part Dog. Part Wolf. All Hero.,An outcast half-wolf risks his life to prevent...,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...","['Family', 'Animation', 'Adventure']",['Balto Collection'],['English'],"['Universal Pictures', 'Amblin Entertainment',...",['United States of America'],['US'],"[{'id': 1994, 'name': 'wolf'}, {'id': 6411, 'n...","['wolf', 'dog-sledding race', 'alaska', 'dog',...","['Kevin Bacon', 'Bob Hoskins', 'Bridget Fonda'...",['Simon Wells'],"[Family, Animation, Adventure]",True,6.689977
33,9598,Babe,en,"[{'iso_639_1': 'en', 'name': 'English'}]",30000000.0,254134910.0,89.0,1995-07-18,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'AU', 'name': 'Australia'}, {'...",tt0112431,14.404764,6.0,756.0,"{'id': 9435, 'name': 'Babe Collection', 'poste...",A little pig goes a long way.,Babe is a little pig who doesn't quite know hi...,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...","['Fantasy', 'Drama', 'Comedy', 'Family']",['Babe Collection'],['English'],"['Universal Pictures', 'Kennedy Miller Product...","['Australia', 'United States of America']","['AU', 'US']","[{'id': 3020, 'name': 'sheep'}, {'id': 4931, '...","['sheep', 'pig', 'affection', 'piglet', 'heroi...","['Christine Cavanaugh', 'Miriam Margolyes', 'D...",['Chris Noonan'],"[Fantasy, Drama, Comedy, Family]",True,5.933653
47,10530,Pocahontas,en,"[{'iso_639_1': 'en', 'name': 'English'}]",55000000.0,346079773.0,81.0,1995-06-14,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",tt0114148,13.280069,6.7,1509.0,"{'id': 136214, 'name': 'Pocahontas Collection'...",An American legend comes to life.,History comes gloriously to life in Disney's e...,"[{'id': 12, 'name': 'Adventure'}, {'

Method 1: step 4 - create the movie recommendation based on the Score

In [96]:
#Sort movies based on score calculated above
qualified_movies = qualified_movies.sort_values('Score', ascending=False)

#Print the top 5 movies
qualified_movies[['Original_Title', 'Vote_Count', 'Vote_Average', 'Popularity','Score']].head(5)

,Original_Title,Vote_Count,Vote_Average,Popularity,Score
5455,千と千尋の神隠し,3968.0,8.3,41.048867,8.194487
1220,Back to the Future,6239.0,8.0,25.778509,7.939554
358,The Lion King,5520.0,8.0,21.605761,7.931907
30046,Inside Out,6737.0,7.9,23.985587,7.846279
17332,Harry Potter and the Deathly Hallows: Part 2,6141.0,7.9,24.990737,7.841200
